# Generate tiled region run file

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: Before running this notebook for the first time, make sure you've coregistered your instrument using the *update coregistration parameters* section of  `1_set_up_toffy.ipynb`. This will ensure your FOVs display correctly on the slide.

This notebook allows you to automatically set up large tiled runs of contiguous FOVs. You can specify the dimensions of the tiled image, for example 5x5, and select specific FOVs you wish to exclude from the final tile.

In [2]:
import sys
sys.path.append('../')

In [3]:
import json
import os
from skimage.io import imread

from toffy.json_utils import read_json_file, write_json_file, split_run_file
from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

## 1. Define the data paths

Set *json_tiling_dir*, the name of the directory to place and write all of your files. This directory should be set as  *C:\\\Users\\\Customer.ION\\\Documents\\\tiled_run_jsons*. <b>Do not change this path</b>.

After running the data paths section, the directory structure should look like:

```
C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons
│   region_corners_path (JSON file defining the position and name of each region to tile)
│   slide_path (image file containing the slide you want to tile over)
```

In [ ]:
# define the name of the directory to use in tiled_run_jsons
json_tiling_dir = 'C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons'

Set a prefix to use for the input and output files to place in *json_tiling_dir*. **All file names must use this prefix**.

The default is *'example_tiled_region'*, but you should change it to something relevant to your study, such as *'BRCA_TMA_1'*.

In [ ]:
# define the prefix for each file
tiled_region_prefix = 'example_tiled_region'

Set the path to the JSON defining the top-left corner FOV of each tiled region.

This file is generated using MIBIcontrol. Give each top-left FOV selected a descriptive name of the tiled region it defines. Move the downloaded file to *json_tiling_dir*, and rename it *{tiled_region_prefix}_region_corners.json*.

In [ ]:
region_corners_path = os.path.join(json_tiling_dir, '%s_region_corners.json' % tiled_region_prefix) 

Set the path to the slide image to take the tiled regions from. This image can be found at *D:\\\Data\\\optical-image*. Move the file to *json_tiling_dir* and rename it *{tiled_region_prefix}_slide.{slide_path_ext}*.

In [ ]:
# Change `slide_path_ext` to '.jpg' or '.png' if needed
slide_path_ext = '.bmp'

slide_path = os.path.join(json_tiling_dir, "%s_slide" % tiled_region_prefix + slide_path_ext)

Set the path in *json_tiling_dir* to write the final run JSON containing the final set of FOVs for each tiled region.

In [ ]:
tiled_region_path = os.path.join(json_tiling_dir, "%s_tiled_region_mapping.json" % tiled_region_prefix)

## 2. Set tiling parameters

Set the dimensions and randomization parameters of each tiled region.

In [ ]:
tiling_params = tiling_utils.set_tiled_region_params(region_corners_path)

## 3. Create the FOVs for each region

FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed. For example, *R1C1* means the FOV is in row 1 and column 1, *R2C1* would be the next FOV down, and *R1C2* would be the next FOV to the right. The top left FOV is *R1C1*, the bottom right FOV would be *R4C5* for a tiled region of 4x5 dimensions. 

In [ ]:
# generate the FOVs in each region
tiled_region_fovs = tiling_utils.generate_tiled_region_fov_list(tiling_params)

## 4. Interactive FOV deselection

The grid defining an ROI may cover more area than you wish to tile. This GUI allows you to interactively delete FOVs from *tiled_region_fovs* prior to saving:

<b>NOTE: if you run the widget and do not need to run deselection, click *Save mapping* immediately to generate the final run file with all the FOVs generated.</b>

1. Click on a FOV to propose it for removal. The border of proposed FOVs will be highlighted. Click again to unpropose the FOV.
2.Once you've selected all the FOVs you want to remove, click *Delete selected FOVs* to remove them. **This action cannot be undone!**
3. You can always remove more FOVs even after saving. Just remember to re-save.
4. After you're done removing all the FOVs, click *Save mapping* to save your truncated set of FOVs to *tiled_region_path*.

In [ ]:
# read the slide image in
slide_data = imread(slide_path)

In [ ]:
%matplotlib widget
_ = tiling_utils.tiled_region_interactive_remap(
    tiled_region_fovs,
    tiling_params,
    slide_data,
    tiled_region_path,
    figsize=(7, 7)
)

## 5. Run JSON adjustment

If you would like to break up your tiled file into smaller JSONs containing a specified amount of FOVs in each, you can do that using the code below. 

`file_split` is a list of values detailing how many FOVs to included in each new file, and must sum to the total number of FOVs in the run file.

In [5]:
file_split = []

split_run_file(json_tiling_dir, os.path.basename(tiled_region_path), file_split)